## Stock management system

In [2]:
import os 
import pymysql
import cryptography

In [4]:
mydb = pymysql.connect(
    host = "localhost",
    user = "root",
    port = 3306,
    password = "morenikeji112"
)
mycursor = mydb.cursor()

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [4]:
mycursor.execute("Create database My_project")
mycursor.execute("Show databases")
for i in mycursor:
    print(i)

('cryptomarket',)
('day12',)
('day15',)
('employeemanagementsystem',)
('information_schema',)
('my_project',)
('mysql',)
('packwaystore',)
('performance_schema',)
('project',)
('sakila',)
('schoolmanagementsystem',)
('schoolmangementsystem',)
('sys',)
('world',)


In [5]:
mycursor.execute("Use My_project")
mycursor.execute("""
        create table stock_management(
        ID int primary key not null,
        Product_name varchar(50) not null,
        cost_price int not null,
        selling_price int not null,
        quantity int not null,
        Avaliability ENUM("Yes","No") not null,
        created_at timestamp default current_timestamp,
        updated_at timestamp default current_timestamp on update current_timestamp)
        """)

0

In [7]:
mycursor.execute("Show columns in stock_management")
for i in mycursor:
    print(i)

('ID', 'int', 'NO', 'PRI', None, '')
('Product_name', 'varchar(50)', 'NO', '', None, '')
('cost_price', 'int', 'NO', '', None, '')
('selling_price', 'int', 'NO', '', None, '')
('quantity', 'int', 'NO', '', None, '')
('Avaliability', "enum('Yes','No')", 'NO', '', None, '')
('created_at', 'timestamp', 'YES', '', 'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED')
('updated_at', 'timestamp', 'YES', '', 'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED on update CURRENT_TIMESTAMP')


In [9]:

# Add the Availability column
try:
    mycursor.execute("ALTER TABLE stock_management ADD COLUMN Availability VARCHAR(10)")
    print("Availability column added successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

mydb.commit()
mycursor.close()
mydb.close()


Availability column added successfully.


In [1]:
mycursor.execute("Use My_project")

# Add new items
def add_item():
    ID = int(input("Enter the product ID: "))
    Product_name = input("What is the name of the product? ")
    cost_price = int(input("What is the cost price? "))
    selling_price = int(input("What is the selling price? "))
    quantity = int(input("Enter the quantity of the product: "))

    # Automatically determine availability
    Availability = "Yes" if quantity > 0 else "No"

    query = """
        INSERT INTO stock_management 
        (ID, Product_name, cost_price, selling_price, quantity, Availability)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    mycursor.execute(query, (ID, Product_name, cost_price, selling_price, quantity, Availability))
    mydb.commit()
    print("Item added successfully.")


## Search for an item in the table with the primary key ID
def search():
    ID = int(input("Enter the product ID: "))
    query = "SELECT * FROM stock_management WHERE ID = %s"
    mycursor.execute(query, (ID,))
    result = mycursor.fetchall()
    if not result:
        print("Record was not found...")
    else:
        print(result)
        

## Updates inventory with the primary key
def update_item():
    ID = int(input("Enter the product ID: "))
    print("1 >> Product name")
    print("2 >> Cost price")
    print("3 >> Selling price")
    print("4 >> Quantity")
    choice = input("Select an option: ")

    if choice == "1":
        Product_name = input("Enter the name of the product: ")
        query = "UPDATE stock_management SET Product_name = %s WHERE ID = %s"
        values = (Product_name, ID)
    elif choice == "2":
        cost_price = int(input("Enter the cost price: "))
        query = "UPDATE stock_management SET cost_price = %s WHERE ID = %s"
        values = (cost_price, ID)
    elif choice == "3":
        selling_price = int(input("Enter the selling price: "))
        query = "UPDATE stock_management SET selling_price = %s WHERE ID = %s"
        values = (selling_price, ID)
    elif choice == "4":
        quantity = int(input("Enter the product quantity: "))
        Availability = "Yes" if quantity > 0 else "No"
        query = "UPDATE stock_management SET quantity = %s, Availability = %s WHERE ID = %s"
        values = (quantity, Availability, ID)
    else:
        print("Invalid selection...")
        return

    mycursor.execute(query, values)
    mydb.commit()
    print("Update was successful.")
    
    
## Delete inventory with primary key
def delete():
    ID = int(input("Enter the product ID: "))
    query = "DELETE FROM stock_management WHERE ID = %s"
    mycursor.execute(query, (ID,))
    mydb.commit()
    print("Deletion successful.")
    
    
## Displays total inventory in an organised format
def display():
    mycursor.execute("SELECT * FROM stock_management")
    records = mycursor.fetchall()

    # Header
    print("\n{:<3} | {:<20} | {:<10} | {:<13} | {:<8} | {:<11}".format(
        "ID", "Product Name", "Cost Price", "Selling Price", "Quantity", "Availability"
    ))
    print("-" * 75)

    # Rows
    for rec in records:
        print("{:<3} | {:<20} | {:<10} | {:<13} | {:<8} | {:<11}".format(
            rec[0], rec[1], rec[2], rec[3], rec[4], rec[5]
        ))


        
## Add stocks with primary key       
def stock_in():
    ID = int(input("Enter the product ID: "))
    quantity_added = int(input("Enter the quantity to add: "))

    # Get current quantity and product name
    mycursor.execute("SELECT Product_name, quantity FROM stock_management WHERE ID = %s", (ID,))
    result = mycursor.fetchone()
    if not result:
        print("Product not found.")
        return

    product_name, current_quantity = result
    new_quantity = current_quantity + quantity_added
    Availability = "Yes" if new_quantity > 0 else "No"

    # Update quantity in database
    mycursor.execute(
        "UPDATE stock_management SET quantity = %s, Availability = %s WHERE ID = %s",
        (new_quantity, Availability, ID)
    )
    mydb.commit()

    print(f"Stock updated successfully for '{product_name}'.")
    print(f"Added {quantity_added} items. New stock level: {new_quantity}")
    
    

## Takes out stock with primary key and displays the remaining stock
def stock_out():
    ID = int(input("Enter the product ID: "))
    quantity_removed = int(input("Enter the quantity to remove: "))

    # Get current quantity and product name
    mycursor.execute("SELECT Product_name, quantity FROM stock_management WHERE ID = %s", (ID,))
    result = mycursor.fetchone()
    if not result:
        print("Product not found.")
        return

    product_name, current_quantity = result
    if quantity_removed > current_quantity:
        print(f"Error: Cannot remove {quantity_removed}. Only {current_quantity} in stock.")
        return

    new_quantity = current_quantity - quantity_removed
    Availability = "Yes" if new_quantity > 0 else "No"

    # Update quantity in database
    mycursor.execute(
        "UPDATE stock_management SET quantity = %s, Availability = %s WHERE ID = %s",
        (new_quantity, Availability, ID)
    )
    mydb.commit()

    print(f"Stock updated successfully for '{product_name}'.")
    print(f"Removed {quantity_removed} items. Remaining stock: {new_quantity}")
    
    
    
## Checks for low stock specifically products under 5   
def low_stock_alert():
    threshold = 5  # You can change this number to set your own low-stock limit
    mycursor.execute("SELECT Product_name, quantity FROM stock_management WHERE quantity < %s", (threshold,))
    low_stock_items = mycursor.fetchall()

    print("\nLow Stock Alert:")
    if low_stock_items:
        for name, qty in low_stock_items:
            print(f"- {name}: {qty} left")
    else:
        print("All products have sufficient stock.")
        
        

## Calculates the total cost value, total selling value and potential profit of inventory and displays it        
def total_inventory_value():
    mycursor.execute("SELECT SUM(cost_price * quantity) FROM stock_management")
    total_cost_value = mycursor.fetchone()[0] or 0

    mycursor.execute("SELECT SUM(selling_price * quantity) FROM stock_management")
    total_selling_value = mycursor.fetchone()[0] or 0

    potential_profit = total_selling_value - total_cost_value

    print("\nInventory Summary:")
    print(f"- Total Cost Value: {total_cost_value}")
    print(f"- Total Selling Value: {total_selling_value}")
    print(f"- Potential Profit: {potential_profit}")


## Front end    
def main():
    print("\nWelcome stock management system")
    while True:
        print("1 << Add Item")
        
        print("2 << Search")
        
        print("3 << Update Item")
        
        print("4 << Delete")
        
        print("5 << Display")
        
        print("6 << Stock in")
        
        print("7 << Stock out")
        
        print("8 << Check for low stock")
        
        print("9 << Total inventory")
        
        print("10 << Exit")
        
        option = input("Select an option: ")
        
        if option == "1":
            add_item()
        elif option == "2":
            search()
        elif option == "3":
            update_item()
        elif option == "4":
            delete()
        elif option == "5":
            display()
        elif option == "6":
            stock_in()
        elif option == "7":
            stock_out()
        elif option == "8":
            low_stock_alert()
        elif option =="9":
            total_inventory_value()
        elif option == "10":
            print("Logout Successful")
            break
        else:
            print("Invalid Option, Please try again")
if __name__ == "__main__":
    main()

NameError: name 'mycursor' is not defined